In [ ]:
# Suppress all warnings and messages
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)
import os, sys

# Redirect stdout during imports
import io
import contextlib
import sys

# Define a context manager to suppress stdout and stderr
@contextlib.contextmanager
def suppress_output():
    # Save the original stdout and stderr
    old_stdout = sys.stdout
    old_stderr = sys.stderr
    # Use StringIO to trap the output
    sys.stdout = io.StringIO()
    sys.stderr = io.StringIO()
    try:
        yield  # Execute the code block inside the with statement
    finally:
        # Restore the original stdout and stderr
        sys.stdout = old_stdout
        sys.stderr = old_stderr

# Suppress output during imports
with suppress_output():
    # Using pip installed dowhy package - no need to modify path
    import dowhy
    from dowhy.do_why import CausalModel
    from rpy2.robjects import r as R
    %load_ext rpy2.ipython

# Suppress R warnings
%R options(warn=-1)
# Suppress R messages
%R suppressPackageStartupMessages(library(Matching))


array(['Matching', 'MASS', 'tools', 'stats', 'graphics', 'grDevices',
       'utils', 'datasets', 'methods', 'base'], dtype='<U9')

In [9]:
%R data(lalonde)
%R -o lalonde


In [10]:
model=CausalModel(
        data = lalonde,
        treatment='treat',
        outcome='re78',
        common_causes='nodegr+black+hisp+age+educ+married'.split('+'))
identified_estimand = model.identify_effect()
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_weighting")
#print(estimate)
print("Causal Estimate is " + str(estimate.value))


INFO:dowhy.causal_identifier:Common causes of treatment and outcome:{'educ', 'black', 'U', 'hisp', 'nodegr', 'age', 'married'}


Model to find the causal effect of treatment treat on outcome re78
{'observed': 'yes'}
{'observed': 'yes'}
{'label': 'Unobserved Confounders', 'observed': 'no'}
There are unobserved common causes. Causal effect cannot be identified.
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y' or 'n'
Please respond with 'y'

KeyboardInterrupt: Interrupted by user

# DoWhy example on the Lalonde dataset

Here we use the Lalonde dataset and apply IPW estimator to it.

In [ ]:
import os, sys
sys.path.append(os.path.abspath("../../"))

import dowhy
from dowhy.do_why import CausalModel
from rpy2.robjects import r as R
%load_ext rpy2.ipython

#%R install.packages("Matching")
%R library(Matching)


/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: MASS

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: ## 
##  Matching (Version 4.9-3, Build Date: 2018-05-03)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##


  warnings.warn(x, RRuntimeWarning)


array(['Matching', 'MASS', 'tools', 'stats', 'graphics', 'grDevices',
       'utils', 'datasets', 'methods', 'base'], 
      dtype='<U9')

## 1. Load the data

In [ ]:
%R data(lalonde)
%R -o lalonde


## Run DoWhy analysis: model, identify, estimate

In [ ]:

model=CausalModel(
        data = lalonde,
        treatment='treat',
        outcome='re78',
        common_causes='nodegr+black+hisp+age+educ+married'.split('+'))
identified_estimand = model.identify_effect()
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_weighting")
#print(estimate)
print("Causal Estimate is " + str(estimate.value))

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:{'black', 'educ', 'U', 'nodegr', 'age', 'married', 'hisp'}


Model to find the causal effect of treatment treat on outcome re78
{'observed': 'yes'}
{'observed': 'yes'}
{'observed': 'no', 'label': 'Unobserved Confounders'}
There are unobserved common causes. Causal effect cannot be identified.
WARN: Do you want to continue by ignoring these unobserved confounders? [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


PropensityScoreWeightingEstimator


INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: re78~treat+black+educ+nodegr+age+married+hisp


Causal Estimate is 1634.98683597


## Sanity check: compare to manual IPW estimate

In [ ]:
df = model._data
ps = df['ps']
y = df['re78']
z = df['treat']

ey1 = z*y/ps / sum(z/ps)
ey0 = (1-z)*y/(1-ps) / sum((1-z)/(1-ps))
ate = ey1.sum()-ey0.sum()
print("Causal Estimate is " + str(ate))

# correct -> Causal Estimate is 1634.9868359746906

Causal Estimate is 1634.98683597
